In [1]:
from dotenv import load_dotenv
load_dotenv()

from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

import os

os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY4"]
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=1,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

In [4]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI="neo4j+s://2d5e8539.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="xn8iCGEj2vymA3-43-57qlL63CD70SthzTE_Mt8QfG0"
NEO4J_DATABASE="neo4j"
AURA_INSTANCEID="2d5e8539"
AURA_INSTANCENAME="Instance01"


from langchain_neo4j import Neo4jGraph
enhanced_graph_db = Neo4jGraph(
    url=NEO4J_URI,
    username="neo4j",
    password=NEO4J_PASSWORD,
    driver_config={
        "max_connection_lifetime": 300,  # 5 minutes
        "keep_alive": True,
        "max_connection_pool_size": 50
    },
    enhanced_schema=True)

graph_db = Neo4jGraph(
    url=NEO4J_URI,
    username="neo4j",
    password=NEO4J_PASSWORD,
    driver_config={
        "max_connection_lifetime": 300,  # 5 minutes
        "keep_alive": True,
        "max_connection_pool_size": 50
    },
    enhanced_schema=False)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
        [
            (
"system",
"""
You are an expert in doing root cause analysis for a provided question. Your job is to generate a list of hypothesis for the given question using a graph database schema.
"""
            ),
            (
"human",
"""
### Graph database schema (Use it for understanding relations)
---
{schema}
---
User Question:
{question}
"""
            ),
        ]
    )
chain = prompt | llm
response = chain.invoke({"schema":graph_db.schema,"question":"Why did my sale decline from 2004 to 2005?"})

In [8]:
print(response.content)

Here's a list of hypotheses to explain the decline in sales from 2004 to 2005, based on the provided graph database schema:

**Overall Sales Volume & Value:**

1.  **Fewer Orders:** The total number of orders placed in 2005 was significantly lower than in 2004.
    *   *(Schema elements: Order {YEAR_ID})*
2.  **Lower Average Order Value:** The average `SALES` generated per order decreased from 2004 to 2005.
    *   *(Schema elements: Order {YEAR_ID}, CONTAINS {SALES})*
3.  **Reduced Quantity Ordered:** The average `QUANTITYORDERED` per product within orders decreased from 2004 to 2005.
    *   *(Schema elements: Order {YEAR_ID}, CONTAINS {QUANTITYORDERED})*
4.  **Lower Average Price Each:** The average `PRICEEACH` at which products were sold decreased from 2004 to 2005.
    *   *(Schema elements: Order {YEAR_ID}, CONTAINS {PRICEEACH})*

**Product Performance:**

5.  **Decline in Specific Product Lines:** One or more high-contributing `PRODUCTLINE`s experienced a significant decline in 